In [1]:
import Random
using Distributions
using Statistics
using DelimitedFiles
using Dates

In [2]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [3]:
function data_collection(initialstate::Vector{Int64},params::Vector{Float64},outcomes,storage,numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG,ntherm)
    endstate,steps,weightsum = direct_gillespie!(initialstate,params,outcomes,transcription_translation_rates!,storage,updatestorage!,rng,numsteps,maxsteps,ntherm)
    return storage,endstate,steps,weightsum
end

function reach_balance_test(initialstate,params,outcomes,numsteps,maxsteps,rng,flux_thres,cov_thres,tries,ntherm)
    statevector = initialstate
    trynum = 0
    thresholdmet = false
    totalsteps = 0
    calcstor = BalanceEqData_nofb()
    fluxcheck = [0,0]
    cbvals = [0,0]
    
    while (trynum < tries) && (!thresholdmet)
        calcstor,statevector,steps,lastsimtime = data_collection(statevector,params,outcomes,calcstor,numsteps,maxsteps,rng,ntherm)
        weightsum = calcstor.weightsum
        fluxcheck = [relative_error(calcstor.meanRpX,calcstor.meanRmX),relative_error(calcstor.meanRpY,calcstor.meanRmY)]
        if (fluxcheck[1] < flux_thres) &&  (fluxcheck[2] < flux_thres)
            cbxx_comp = cov_balance_comp(calcstor.CXRmX/weightsum,calcstor.CXRmX/weightsum,
            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/weightsum,calcstor.CXRpX/weightsum) 
            cbyy_comp = cov_balance_comp(calcstor.CYRmY/weightsum,calcstor.CYRmY/weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/weightsum,calcstor.CYRpY/weightsum) 
            cbyx_comp = cov_balance_comp(calcstor.CYRmX/weightsum,calcstor.CXRmY/weightsum,
                            0,calcstor.CYRpX/weightsum,calcstor.CXRpY/weightsum) 
            cbvals = [cbxx_comp,cbyy_comp,cbyx_comp]
            thresholdmet = all(cbvals .< cov_thres)               
        end
        trynum += 1
        totalsteps += steps
        if maxsteps>100000
            maxsteps = maxsteps ÷ 2
        end
    end
return calcstor,totalsteps,trynum,fluxcheck,cbvals
end

reach_balance_test (generic function with 1 method)

Run some number of simulations, storing the used parameters+simulation constraints. Save each simulation's output into a big CSV. Finally, go through the results and calculate the perturbation data

Generate truly independent perts: cartesian product of all the values you want to multiply

In [9]:
lambdas = [1,1.1,0.9]
betaxs = [1,0.8,1.2]
gammas = [1,1.2,0.8]
betays = [1]

independentperts = [[d_l,d_bx,d_g,d_by] for d_l in lambdas for d_bx in betaxs for d_g in gammas for d_by in betays]

27-element Vector{Vector{Float64}}:
 [1.0, 1.0, 1.0, 1.0]
 [1.0, 1.0, 1.2, 1.0]
 [1.0, 1.0, 0.8, 1.0]
 [1.0, 0.8, 1.0, 1.0]
 [1.0, 0.8, 1.2, 1.0]
 [1.0, 0.8, 0.8, 1.0]
 [1.0, 1.2, 1.0, 1.0]
 [1.0, 1.2, 1.2, 1.0]
 [1.0, 1.2, 0.8, 1.0]
 [1.1, 1.0, 1.0, 1.0]
 ⋮
 [0.9, 1.0, 1.0, 1.0]
 [0.9, 1.0, 1.2, 1.0]
 [0.9, 1.0, 0.8, 1.0]
 [0.9, 0.8, 1.0, 1.0]
 [0.9, 0.8, 1.2, 1.0]
 [0.9, 0.8, 0.8, 1.0]
 [0.9, 1.2, 1.0, 1.0]
 [0.9, 1.2, 1.2, 1.0]
 [0.9, 1.2, 0.8, 1.0]

In [10]:
folder = "constx_sims_largey_"*Dates.format(now(),"yyyy-udd_HHMM")*"/"
#mkdir(folder)
is = [10,10]
outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
numsteps = Int(10e6)
maxsteps = Int(10e8)
fluxt = 0.02
covt = 0.05
tries = 40
ntherm = Int(10e5)

paramsets = [
    [25.,1.,20.,0.5],
    [10.,1.,50.,1.],
    [50.,1.,50.,1.]]
for (k,params) in enumerate(paramsets)
    i = k
    println("Paramset $(paramsets)")
    pertparams = copy(params)
    open(folder*"paramset_$(i)_settings.tsv", "w") do io
        titles = ["params","outcomes","initialstate","numsteps","maxsteps","flux thres","cov thres","max tries","thermalisation"]
        data = [params,outcomes,is,numsteps,maxsteps,fluxt,covt,tries,ntherm]
        writedlm(io,reshape(titles,1,length(titles)))
        writedlm(io,reshape(data,1,length(data)))
    end;
    open(folder*"paramset_$(i)_simresults.tsv", "w") do io
        titles = ["sim #","time","steps","flux thres","cov thres","<x>","<y>","Var(x)","Var(y)","cov(x,y)","usedparams"]
        writedlm(io, reshape(titles,1,length(titles)))
    end;
    for (j,perturbation) in enumerate(independentperts)
        statevector = copy(is)
        pertparams .*= perturbation
        rng = Random.Xoshiro()
        calcstor,totalsteps,trynum,fluxcheck,cbvals = reach_balance_test(statevector,pertparams,outcomes,numsteps,maxsteps,rng,fluxt,covt,tries,ntherm)
        open(folder*"paramset_$(i)_simresults.tsv", "a+") do io
            data = [j,calcstor.weightsum,totalsteps,fluxcheck,cbvals,calcstor.meanx,calcstor.meany,calcstor.VX/calcstor.weightsum,calcstor.VY/calcstor.weightsum,calcstor.CXY/calcstor.weightsum,pertparams]
            writedlm(io, reshape(data,1,length(data)))
        end;
        pertparams ./= perturbation
    end
end

Paramset [[25.0, 1.0, 2.0, 1.0], [10.0, 1.0, 20.0, 5.0], [50.0, 2.0, 1.0, 1.0]]


InterruptException: InterruptException: